# Este notebook faz parte da reportagem [Ceará é o 3° Estado do Nordeste e 7° do Brasil com mais obras do Novo PAC](https://mais.opovo.com.br/reportagens-especiais/novo-pac/2023/09/24/ceara-e-o-3-estado-do-nordeste-e-7-do-brasil-com-mais-obras-do-novo-pac.html?collection=reportagens_de_dados)

Para esta reportagem, foram usados dados do NOVO PAC / Casa Civil - Governo Federal. Foi analisada a tabela de obras por estado e coletados, de maneira automatizada, os valores de cada eixo de investimento. Os valores previstos para cada obra não foram disponibilizados e os montantes para cada eixo/subeixo não foram estruturados em formato aberto, impossibilitando uma consulta cidadã detalhada da distribuição dos investimentos.

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
codigos_rg_uf = pd.read_csv('bases_de_dados/cod_sg_uf.csv')

In [3]:
# Dados públicos do mapa de obras
dataset_novo_pac = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR5_5AFiquBfta4UtwTWt89xa9MTz0WjXNwCLOgPv-ZVDWuA9TvioKeH2AXgC894A/pub?gid=565983705&single=true&output=csv')

In [4]:
dataset_novo_pac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12521 entries, 0 to 12520
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   EIXO                                12521 non-null  object
 1   SUBEIXO                             12521 non-null  object
 2   UF                                  12521 non-null  object
 3   MUNICÍPIO                           12521 non-null  object
 4   NOME                                12521 non-null  object
 5   MODALIDADE                          12521 non-null  object
 6   OBRA, PROJETO, ESTUDO, EQUIPAMENTO  12521 non-null  object
dtypes: object(7)
memory usage: 684.9+ KB


In [5]:
dataset_novo_pac.head()

,EIXO,SUBEIXO,UF,MUNICÍPIO,NOME,MODALIDADE,"OBRA, PROJETO, ESTUDO, EQUIPAMENTO"
0,Água para Todos,Abastecimento de água,AC,Rio Branco,Implantação do SAA Cidade do Povo​​​,"Retomada, conclusão e novas obras",Obra
1,Água para Todos,Abastecimento de água,BA,Amélia Rodrigues,Ampliação do SAA-ETA ​,"Retomada, conclusão e novas obras",Obra
2,Água para Todos,Abastecimento de água,BA,"Belo Campo, Vitória da Conquista, Tremedal",Ampliação do SAA ​,"Retomada, conclusão e novas obras",Obra
3,Água para Todos,Abastecimento de água,BA,Camaçari,Ampliação do SAA​,"Retomada, conclusão e novas obras",Obra
4,Água para Todos,Abastecimento de água,BA,Luís Eduardo Magalhães,Ampliação do SAA - 1ª Etapa​,"Retomada, conclusão e novas obras",Obra


In [6]:
npac = dataset_novo_pac.copy()

In [7]:
npac.shape

(12521, 7)

In [8]:
npac['OBRA, PROJETO, ESTUDO, EQUIPAMENTO'].value_counts()

Obra                12024
obra                  164
Projeto               138
Equipamento            92
Estudo                 69
projeto                16
Estudo/Projeto​         8
estudo                  6
Obra/Equipamento        4
Name: OBRA, PROJETO, ESTUDO, EQUIPAMENTO, dtype: int64

In [9]:
npac['MODALIDADE'].value_counts()

Retomada e conclusão de obras - creche, escola, quadra e cobertura de quadra    2630
Faixa 2                                                                         2104
Faixa 1                                                                         1984
Retomada e conclusão                                                            1516
Faixa 3                                                                         1409
                                                                                ... 
Seleção - Drenagem                                                                 1
Seleção - Centros de Atenção Psicossocial                                          1
Hidrelétrica​                                                                      1
Reforma e recuperação de acervos da Biblioteca Nacional - FBN                      1
Seleção - Encostas                                                                 1
Name: MODALIDADE, Length: 149, dtype: int64

In [10]:
npac.groupby('EIXO').size().reset_index()

,EIXO,0
0,Cidades Sustentáveis e Resilientes,7493
1,"Educação, Ciência e Tecnologia",2721
2,Inclusão Digital e Conectividade,251
3,Infraestrutura Social e Inclusiva,192
4,Inovação para a indústria da defesa,16
5,Saúde,460
6,Transição e Segurança Energética,589
7,Transporte Eficiente e Sustentável,595
8,Água para Todos,204


In [11]:
npac.loc[npac.UF=="CE"].groupby('EIXO').size().reset_index()

,EIXO,0
0,Cidades Sustentáveis e Resilientes,298
1,"Educação, Ciência e Tecnologia",151
2,Inclusão Digital e Conectividade,8
3,Infraestrutura Social e Inclusiva,10
4,Saúde,24
5,Transição e Segurança Energética,55
6,Transporte Eficiente e Sustentável,11
7,Água para Todos,17


In [12]:
npac.loc[npac.UF.str.contains("CE")].groupby('EIXO').size().reset_index()

,EIXO,0
0,Cidades Sustentáveis e Resilientes,298
1,"Educação, Ciência e Tecnologia",151
2,Inclusão Digital e Conectividade,9
3,Infraestrutura Social e Inclusiva,10
4,Saúde,24
5,Transição e Segurança Energética,57
6,Transporte Eficiente e Sustentável,15
7,Água para Todos,22


In [13]:
npac.loc[(npac.UF.str.contains("CE")) & (npac.UF!="CE")].groupby('EIXO').size().reset_index()

,EIXO,0
0,Inclusão Digital e Conectividade,1
1,Transição e Segurança Energética,2
2,Transporte Eficiente e Sustentável,4
3,Água para Todos,5


In [14]:
resumo_eixo_subeixo = npac.groupby(['EIXO','SUBEIXO','UF']).size().reset_index()
resumo_eixo_subeixo.columns = ['Eixo','Subeixo','UF','Nº de Obras']
resumo_eixo_subeixo
resumo_eixo_subeixo.to_csv('resumo_eixo_subeixo_uf.csv', index=False)

In [15]:
# obras_uf = npac.loc[~((npac.UF.str.contains('/')) | (npac.UF.str.contains(' '))) & (npac.UF != "Nacional")].groupby(['UF']).size().reset_index()
# obras_uf['EIXO'] = 'Todos os eixos'
obras_eixos_uf = npac.loc[~((npac.UF.str.contains('/')) | (npac.UF.str.contains(' '))) & (npac.UF != "Nacional")].groupby(['UF','EIXO']).size().reset_index()

# obras_eixos_uf = pd.concat([obras_uf,obras_eixos_uf])
# obras_eixos_uf.columns = ['UF','Qtd','Eixo']
obras_eixos_uf = pd.merge(obras_eixos_uf, codigos_rg_uf, left_on='UF', right_on="SIGLA")
obras_eixos_uf.to_csv('obras_eixos_uf.csv', index=False)
obras_eixos_uf

,UF,EIXO,0,COD,NOME,SIGLA,REGIAO
0,AC,Cidades Sustentáveis e Resilientes,17,12,Acre,AC,Norte
1,AC,"Educação, Ciência e Tecnologia",22,12,Acre,AC,Norte
2,AC,Inclusão Digital e Conectividade,7,12,Acre,AC,Norte
3,AC,Infraestrutura Social e Inclusiva,1,12,Acre,AC,Norte
4,AC,Saúde,8,12,Acre,AC,Norte
...,...,...,...,...,...,...,...
200,TO,Inclusão Digital e Conectividade,7,17,Tocantins,TO,Norte
201,TO,Saúde,7,17,Tocantins,TO,Norte
202,TO,Transição e Segurança Energética,1,17,Tocantins,TO,Norte
203,TO,Transporte Eficiente e Sustentável,8,17,Tocantins,TO,Norte


In [16]:
obras_eixos_uf.groupby('REGIAO')[0].sum()

REGIAO
Centro-Oeste    1080
Nordeste        4337
Norte           1245
Sudeste         3617
Sul             1995
Name: 0, dtype: int64

In [17]:
codigos_rg_uf.sort_values(by='SIGLA')

,COD,NOME,SIGLA,REGIAO
1,12,Acre,AC,Norte
13,27,Alagoas,AL,Nordeste
2,13,Amazonas,AM,Norte
5,16,Amapá,AP,Norte
15,29,Bahia,BA,Nordeste
9,23,Ceará,CE,Nordeste
26,53,Distrito Federal,DF,Centro-Oeste
17,32,Espírito Santo,ES,Sudeste
25,52,Goiás,GO,Centro-Oeste
7,21,Maranhão,MA,Nordeste


In [18]:
#Existem Obras em diferentes estados/municípios

In [19]:
npac.UF.value_counts().reset_index().tail(2)

,index,UF
115,AC/RO,1
116,CE/PE,1


In [20]:
npac.MUNICÍPIO.nunique()

6310

In [21]:
# Cidades do Ceará:

In [22]:
npac.loc[(npac.UF == "CE") & (npac['MUNICÍPIO'] != '-')]

,EIXO,SUBEIXO,UF,MUNICÍPIO,NOME,MODALIDADE,"OBRA, PROJETO, ESTUDO, EQUIPAMENTO"
9,Água para Todos,Abastecimento de água,CE,Camocim,Ampliação do SAA​​​,"Retomada, conclusão e novas obras",Obra
10,Água para Todos,Abastecimento de água,CE,Caucaia,Implantação do SAA da Vila e Praias do Cumbuco,"Retomada, conclusão e novas obras",Obra
11,Água para Todos,Abastecimento de água,CE,Caucaia,Ampliação do SAA,"Retomada, conclusão e novas obras",Obra
12,Água para Todos,Abastecimento de água,CE,Jijoca de Jericoacoara,Ampliação do SAA da Vila de Jericoacoara,"Retomada, conclusão e novas obras",Obra
13,Água para Todos,Abastecimento de água,CE,"São Benedito, Ubajara, Graça, Carnaubal, Pacuj...",Ampliação do SAA Ibiapaba,"Retomada, conclusão e novas obras",Obra
...,...,...,...,...,...,...,...
11488,Transição e Segurança Energética,Geração de Energia,CE,Icapuí​,Mutamba IV - 33 MW​,Eólica​,Obra
11949,Transporte Eficiente e Sustentável,Aeroportos,CE,Aracati,Implantação de cerca operacional,Retomada e Conclusão,Obra
11950,Transporte Eficiente e Sustentável,Aeroportos,CE,Juazeiro do Norte,Juazeiro do Norte,Concessões existentes,Obra
12134,Transporte Eficiente e Sustentável,Portos,CE,Fortaleza,Fortaleza/CE: ARRENDAMENTO - TERGRAN TERMINAIS...,Arrendamento,obra


In [23]:
resumo_ce = npac.loc[(npac.UF == "CE") & (~npac['MUNICÍPIO'].str.contains(','))].copy()
eixos = list(resumo_ce.EIXO.unique())

resumo_ce['MUNICÍPIO'] = resumo_ce['MUNICÍPIO'].apply(lambda x: x.title().replace(' Do ',' do ').replace('\u200b',''))
resumo_ce = resumo_ce.groupby(['MUNICÍPIO','EIXO']).size().reset_index()

resumo_ce.columns = ['Cidade','Eixo','Qtd']
resumo_ce = resumo_ce.pivot(index='Cidade',columns='Eixo').fillna(0).astype(int).reset_index().droplevel(level=0, axis=1)
resumo_ce.columns = ['Cidade'] + eixos

resumo_ce['Todos os Eixos'] = resumo_ce[eixos].sum(axis=1)

In [24]:
resumo_ce.loc[resumo_ce.Cidade.str.contains('Cauc')]

,Cidade,Água para Todos,Cidades Sustentáveis e Resilientes,"Educação, Ciência e Tecnologia",Inclusão Digital e Conectividade,Infraestrutura Social e Inclusiva,Saúde,Transição e Segurança Energética,Transporte Eficiente e Sustentável,Todos os Eixos
40,Caucaia,5,1,0,0,1,1,0,2,10


In [26]:
path_geom = 'bases_de_dados/gdf_cidades_ce.geojson'
gdf_cidades = gpd.read_file(path_geom)
gdf_cidades.head(2)

,Cidade,População em 2010,População 2010 (Alterações de Limites até 2022),População em 2022,cod_mun,Variação (%),Resultado,geometry
0,Abaiara,10496,10456,10038,101.0,-4.00,Diminuiu,"POLYGON ((-38.96990 -7.42730, -38.96900 -7.424..."
1,Acarape,15338,13661,14027,150.0,2.68,Aumentou,"POLYGON ((-38.61880 -4.16350, -38.61990 -4.163..."


In [27]:
merge=pd.merge(resumo_ce,gdf_cidades, on='Cidade', how='outer', indicator=True)

In [28]:
merge._merge.value_counts()

both          165
right_only     19
left_only      10
Name: _merge, dtype: int64

In [29]:
gdf_cidades.loc[gdf_cidades.Cidade.str.contains('Caucai')].iloc[0].Cidade == resumo_ce.loc[resumo_ce.Cidade.str.contains('Caucai')].iloc[0].Cidade

True

In [30]:
print(resumo_ce.loc[resumo_ce.Cidade.str.contains('Caucai')].iloc[0].Cidade)

Caucaia


In [31]:
resumo_ce.loc[resumo_ce.Cidade.str.contains('Caucaia') ]

,Cidade,Água para Todos,Cidades Sustentáveis e Resilientes,"Educação, Ciência e Tecnologia",Inclusão Digital e Conectividade,Infraestrutura Social e Inclusiva,Saúde,Transição e Segurança Energética,Transporte Eficiente e Sustentável,Todos os Eixos
40,Caucaia,5,1,0,0,1,1,0,2,10


In [32]:
merge.loc[merge._merge == 'left_only'][['Cidade','Todos os Eixos', 'geometry']]

,Cidade,Todos os Eixos,geometry
0,-,28.0,None
43,Colatina,1.0,None
47,Cristino Castro,2.0,None
52,Ererê,1.0,None
80,Itapagé,1.0,None
86,Jaguaretana,5.0,None
91,Jijoca De Jericoacoara,3.0,None
94,Lavras Da Mangabeira,2.0,None
113,Mucuripe,1.0,None
125,Panorama,5.0,None


In [33]:
merge.loc[merge._merge == 'right_only'][['Cidade','Todos os Eixos', 'geometry']]

,Cidade,Todos os Eixos,geometry
175,Altaneira,NaN,"POLYGON ((-39.68060 -7.01610, -39.67800 -7.011..."
176,Aratuba,NaN,"POLYGON ((-38.97040 -4.36820, -38.97360 -4.362..."
177,Arneiroz,NaN,"POLYGON ((-39.88860 -6.09740, -39.89370 -6.096..."
178,Catarina,NaN,"POLYGON ((-39.78930 -6.20650, -39.79460 -6.208..."
179,Cedro,NaN,"POLYGON ((-38.92160 -6.56670, -38.92170 -6.565..."
180,Choró,NaN,"POLYGON ((-38.97810 -4.66790, -38.97690 -4.665..."
181,Forquilha,NaN,"POLYGON ((-40.18870 -3.95870, -40.18880 -3.956..."
182,Guaramiranga,NaN,"POLYGON ((-38.89120 -4.27700, -38.89100 -4.273..."
183,Hidrolândia,NaN,"POLYGON ((-40.25440 -4.47210, -40.25430 -4.469..."
184,Itaiçaba,NaN,"POLYGON ((-37.61630 -4.71800, -37.64480 -4.716..."


In [34]:
merge.loc[merge.Cidade.str.contains('Mucu')]

,Cidade,Água para Todos,Cidades Sustentáveis e Resilientes,"Educação, Ciência e Tecnologia",Inclusão Digital e Conectividade,Infraestrutura Social e Inclusiva,Saúde,Transição e Segurança Energética,Transporte Eficiente e Sustentável,Todos os Eixos,População em 2010,População 2010 (Alterações de Limites até 2022),População em 2022,cod_mun,Variação (%),Resultado,geometry,_merge
113,Mucuripe,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only


In [35]:
#Coleta eixos:

In [36]:
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [37]:
page_info=urlopen('https://www.gov.br/casacivil/novopac')

soup = BeautifulSoup(page_info, 'html.parser')

eixos = soup.findAll("div", {"class": "cover-banner-tile tile-content"})

In [38]:
len(eixos)

14

In [39]:
valores_por_eixo = []

for e in eixos[4:13]:
    link = e.findAll("a")[0]['href']
    eixo = link.replace('https://www.gov.br/casacivil/pt-br/acoes-e-programas/novo-pac/','')
    soup_eixo = BeautifulSoup(urlopen(f'https://www.gov.br/casacivil/novopac/{eixo}'), 'html.parser')
    
    divs = soup_eixo.findAll("div", {"class": "cover-richtext-tile tile-content"})
    valores = []
    for d in divs[:5]:
        teste = str(d)
        if "R$" in teste:
            valores.append(teste.split("R$</span>")[1].split(">")[1].split("</span")[0].replace(',','.'))
            
    valores_por_eixo.append({"Eixo": eixo, "Total": valores[0], "2023-2026": valores[1], "Pós 2026": valores[2]})

In [40]:
resumo_eixos = pd.DataFrame(valores_por_eixo)
resumo_eixos.Eixo = resumo_eixos.Eixo.apply(lambda x: x.replace('-',' ').capitalize())

resumo_eixos.to_csv('resultado/valores_por_eixo.csv', index=False)

In [41]:
resumo_eixos

,Eixo,Total,2023-2026,Pós 2026
0,Transporte eficiente e sustentavel,349.1,220.9,128.2
1,Infraestrutura social inclusiva,2.4,2.1,0.3
2,Cidades sustentaveis e resilientes,609.7,557.1,52.6
3,Agua para todos,30.5,25.6,4.9
4,Inclusao digital e conectividade,27.9,20.3,7.6
5,Transicao e seguranca energetica,565.4,417.5,147.9
6,Inovacao para a industria da defesa,52.8,27.8,25
7,Educacao ciencia e tecnologia,45,36.7,8.3
8,Saude,30.5,29.3,1.2


In [42]:
resumo_eixos.Total.astype(float).sum()

1713.3

In [43]:
resumo_eixos['2023-2026'].astype(float).sum()

1337.3

In [44]:
resumo_eixos['Pós 2026'].astype(float).sum()

376.0

In [45]:
pd.DataFrame(valores_por_eixo)

,Eixo,Total,2023-2026,Pós 2026
0,transporte-eficiente-e-sustentavel,349.1,220.9,128.2
1,infraestrutura-social-inclusiva,2.4,2.1,0.3
2,cidades-sustentaveis-e-resilientes,609.7,557.1,52.6
3,agua-para-todos,30.5,25.6,4.9
4,inclusao-digital-e-conectividade,27.9,20.3,7.6
5,transicao-e-seguranca-energetica,565.4,417.5,147.9
6,inovacao-para-a-industria-da-defesa,52.8,27.8,25
7,educacao-ciencia-e-tecnologia,45,36.7,8.3
8,saude,30.5,29.3,1.2
